# Regresión Lineal

### Laboratorio de Datos, IC - FCEN - UBA - 1er. Cuatrimestre 2024



### Nuevas librerías

Utilizaremos el módulo `scikit-learn` y `formulaic` de Python. Para instalarlos, correr:

In [ ]:
# !pip install scikit-learn
# !pip install formulaic

Importamos los módulos de siempre, las herramientas de modelos lineales y las medidas de desempeño del modelo de `scikit-learn`:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so

# Para plantear el modelo lineal
from formulaic import Formula   

# Herramientas de modelos lineales
from sklearn import linear_model  

# Medidas de desempeño
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error    

### Modelo de Regresión Lineal

Queremos utilizar un modelo lineal:
$$ Y = \beta_0 + \beta_1 X $$
para intentar predecir valores continuos.

### Visualización

Con lo que hemos visto las clases anteriores, visualizar la recta que mejor aproxima a los datos en el sentido de cuadrados mínimos es relativamente sencillo con `seaborn objects`, ya que `seaborn` calcula automáticamente los valores de $\beta_0$ y $\beta_1$.

Utilizaremos el dataset _tips_ de `seaborn` y analizaremos la relación entre lo que costó la comida y la cantidad de propina.

In [ ]:
datos = sns.load_dataset('tips')

Visualizamos los datos con `so.Plot` (repasar clase de visualización)

In [ ]:
# Graficamos la relacion entre las variables 

(
    so.Plot(data=datos, x='total_bill', y='tip')
    .add(so.Dot())
)

Para visualizar la recta de la regresión, agregamos una línea al gráfico (`so.Line()`) junto a `so.PolyFit(1)`:

In [ ]:
(
    so.Plot(data=datos, x='total_bill', y='tip')
    .add(so.Dot())
    .add(so.Line(color='red', linewidth=3), so.PolyFit(1), label='Regresion lineal')    # Agregamos una etiqueta para la leyenda del grafico
    .label(title='Datos de propinas', x='Precio de la comida', y='Propina')    # Agregamos la leyenda, un titulo y le cambiamos el nombre a los ejes
)

### Cálculo de coeficientes y predicciones

Desafortunamente, `seaborn` no nos devuelve los valores de $\beta_0$ y $\beta_1$, que son importantes en la interpretación del resultado.

Hemos visto que podemos calcular los coeficientes de la regresión lineal como:

 $$
 \begin{array}{rl}
      \hat{\beta}_1 = & \dfrac{\displaystyle\sum_{i=1}^n(x_i - \bar{x})(y_i - \bar{y})}{\displaystyle\sum_{i=1}^n(x_i - \bar{x})^2}  \\[1em]
      \hat{\beta}_0 = & \bar{y} - \hat{\beta}_1\bar{x}
 \end{array}
 $$

 pero también podemos utilizar `scikit-learn` y `formulaic`.

Como queremos predecir la propina según el precio de la comida usando una función lineal, la fórmula de Wilkinson es:
$$tip \sim total\_bill$$

In [ ]:
# Obtenemos las matrices del modelo
y, X = Formula('tip ~ total_bill').get_model_matrix(datos)

In [ ]:
# Inicializamos el modelo de regresión.
modelo = linear_model.LinearRegression(fit_intercept=False) # RECORDAR USAR fit_intercept = False

# Realiza el ajuste
modelo.fit(X, y)

# Para obtener los valores de beta_1 y beta_0 como valores numericos
beta = modelo.coef_
beta_0 = beta[0][0]
beta_1 = beta[0][1]
print('Beta_0: ', beta_0)
print('Beta_1: ', beta_1)

Entonces, la recta que mejor aproxima a los datos es (redondeando):
$$ Y = 0.92 + 0.105 X$$

Una interpretación que podemos darle a este resultado es que, por cada peso que costó la comida, se dejan de propina 0.105 pesos (o sea, alrededor de 10 centavos)

Con los valores de $\beta_0$ y $\beta_1$ podemos predecir cuanto será la propina según el valor de la comida. Supongamos que queremos predecir la propina que se deja por una cuenta \\$35. Utilizamos el método `predict()` del modelo.

<span style="color:red">**EL MÉTODO .predict() SOLO ADMITE ADMITE DATAFRAMES (O MATRICES).**</span>

In [ ]:
# Creamos un array con los valores a predecir
x_a_predecir = np.array([35])

# Creamos un DataFrame con los valores de X para los que queremos predecir
# Debe tener una columna de 1's que sea el Intercept (término independiente)
dataframe_a_predecir = pd.DataFrame({'Intercept': np.ones(x_a_predecir.shape), 'total_bill': x_a_predecir})

# Aplicamos la fórmula del modelo
modelo.predict(dataframe_a_predecir)

In [ ]:
# Para devolver directamente el numero agregamos .item()
modelo.predict(dataframe_a_predecir).item()

También podemos hacer directamente la cuenta porque ya sabemos cuanto valen $\beta_1$ y $\beta_0$:

In [ ]:
beta_1 * 35 + beta_0

Podemos predecir varios valores de una:

In [ ]:
# Creamos un array con los valores a predecir
x_a_predecir = np.array([35, 60])

# Creamos un DataFrame con los valores de X para los que queremos predecir
# Debe tener una columna de 1's que sea el Intercept (término independiente)
dataframe_a_predecir = pd.DataFrame({'Intercept': np.ones(x_a_predecir.shape), 'total_bill': x_a_predecir})

# Aplicamos la fórmula del modelo
modelo.predict(dataframe_a_predecir)

Si queremos calcular los valores predichos por el modelo $\hat{y}_i$ para todos nuestros $x_i$:

In [ ]:
y_pred = modelo.predict(X)
display(y_pred)

### ¿Qué tan bueno es el modelo?

Finalmente, el bueno de `scikit-learn` nos calcula el coeficiente de determinación $R^ 2$: primero van los datos observados ( $y$ ) y luego los datos predichos ( $\hat{y}$ )

In [ ]:
y_pred = modelo.predict(X)
r2_score(datos['tip'], y_pred)

De manera análoga podemos calcular el error cuadrático medio (ECM):

In [ ]:
mean_squared_error(datos['tip'], y_pred)

Y la raíz del error cuadrático medio:

In [ ]:
root_mean_squared_error(datos['tip'], y_pred)

Esto último podría interpretarse (informalmente) como que al usar el modelo para predecir cuánta propina dejo, en
promedio voy a cometer un error en el margen de $\pm 1.01785$ pesos.